<a href="https://colab.research.google.com/github/GzpTez0514/-/blob/main/Pytorch%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A008_Pytorch%E5%AE%9E%E7%8E%B0%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
X = torch.tensor(1., requires_grad=True) # requires_grad 表示允许对X进行梯度计算
y = X ** 2

grad = torch.autograd.grad(y, X) # 这里返回的是在函数y = X ** 2上，X = 1时的导数值
print(grad)


(tensor(2.),)


In [4]:
# 对于单层神经网络，autograd.grad会非常有效。但深层神经网络就不太适合使用grad函数了
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(420)
X = torch.rand((500, 20), dtype=torch.float32) * 100
y = torch.randint(low=0, high=3, size=(500, 1), dtype=torch.float32)

# 定义神经网络的架构
class Model(nn.Module):
  def __init__(self, in_features=10, out_features=2):
    super().__init__()
    self.linear1 = nn.Linear(in_features, 13, bias=True)
    self.linear2 = nn.Linear(13, 8, bias=True)
    self.output = nn.Linear(8, out_features, bias=True)
    
  def forward(self, X):
    z1 = self.linear1(X)
    sigma1 = torch.relu(z1)
    z2 = self.linear2(sigma1)
    sigma2 = torch.sigmoid(z2)
    z3 = self.output(sigma2)
  # sigma3 = F.softmax(z3, dim=1)
    return z3

input_ = X.shape[1] # 特征的数目
output_ = len(y.unique()) # 分类的数目

# 实例化神经网络类
torch.manual_seed(420)
net = Model(in_features=input_, out_features=output_)
# 正向传播
zhat = net.forward(X)
# 定义损失函数
criterion = nn.CrossEntropyLoss()
# 对于打包好的CrossEntropyLoss而言，只需要输入zhat
loss = criterion(zhat, y.reshape(500).long())
print(loss)
print(net.linear1.weight.grad) # 不会返回任何值
# 反向传播，backward是任意损失函数类都可以调用的方法，对任意损失函数，backward都会求解其中全部w的梯度
loss.backward()
print(net.linear1.weight.grad) # 返回响应的梯度

# 与可以重复进行的正向传播不同，一次正向传播后，反向传播只能进行一次
# 如果希望可以重复进行反向传播，可以在第一次进行反向传播的时候加上参数retain_graph
loss.backward(retain_graph=True)
loss.backward()

tensor(1.1057, grad_fn=<NllLossBackward0>)
None
tensor([[ 3.3727e-04,  8.3354e-05,  4.0867e-04,  4.3058e-05,  1.4551e-04,
          6.5092e-05,  3.7088e-04,  2.8794e-04,  1.0495e-04,  4.7446e-05,
          8.8153e-05,  1.6899e-04,  1.0251e-04,  3.6197e-04,  1.2129e-04,
          7.2405e-05,  1.4479e-04,  4.9114e-06,  1.0770e-04,  9.5156e-05],
        [ 8.2042e-03,  2.1974e-02,  2.1073e-02,  1.3896e-02,  2.2161e-02,
          1.5936e-02,  1.6537e-02,  2.0259e-02,  1.9655e-02,  1.4728e-02,
          1.9212e-02,  2.0086e-02,  1.8295e-02,  8.4132e-03,  1.8036e-02,
          1.9979e-02,  2.0966e-02,  2.4730e-02,  9.3876e-03,  1.7475e-02],
        [ 9.1603e-03,  2.4275e-02,  2.3446e-02,  2.0096e-02,  2.5360e-02,
          1.7406e-02,  3.2555e-02,  2.2461e-02,  3.6793e-03,  2.7445e-02,
          2.1181e-02,  2.7724e-02,  1.7115e-02,  1.6943e-02,  1.7249e-02,
          3.3173e-02,  1.5115e-02,  3.0874e-02,  1.8391e-02,  2.4201e-02],
        [-2.8595e-04,  1.2968e-03,  1.3652e-03, -5.6692e-05, 